In [5]:
import threading

threading.active_count() #有幾個active的thread

threading.enumerate() #這些thread分別是什麼
# [<_MainThread(MainThread, started 140736011932608)>, <Thread(SockThread, started daemon 123145376751616)>]


[<_MainThread(MainThread, started 140632979371776)>,
 <Thread(Thread-2, started daemon 140632731416320)>,
 <Heartbeat(Thread-3, started daemon 140632723023616)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140632697845504)>,
 <ParentPollerUnix(Thread-1, started daemon 140632688928512)>]

In [7]:
threading.current_thread() #目前正在運行的thread
# <_MainThread(MainThread, started 140736011932608)>

<_MainThread(MainThread, started 140632979371776)>

In [8]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

def main():
    thread = threading.Thread(target=thread_job)   #添加一個thread
    thread.start()  # 執行thread
    
if __name__ == '__main__':
    main()

This is a thread of <Thread(Thread-4, started 140632680273664)>


In [18]:
import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1) # 任务间隔0.1s
    print("T1 finish\n")

added_thread = threading.Thread(target=thread_job, name='T1')
added_thread.start()
#added_thread.join() #join後 要等待T1結束後,才接下去作另一個功能
print("all done\n")

all done
T1 start


T1 finish



In [34]:
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 开启T1
#thread_1.join() # notice the difference!
thread_2.start() # 开启T2
#thread_2.join()
#print(threading.enumerate())
print("all done\n")

all done
T1 start
T2 start



T2 finish

T1 finish



In [2]:
import threading
import time

from queue import Queue

def job(l,q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)

def multithreading():
    q = Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i], q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
          results.append(q.get())
    print(results)

if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


In [4]:
#GIL (global interpreter lock),多個thread其實一次只執行一個,只是互相切換
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.28827476501464844
1999998000000
multithreading:  0.2793617248535156


In [31]:
#當兩個thread需要用到共享的內存,需要lock功能
#若不用lock,此案例會job1與job2同時運行,A會亂跳
import threading
def job1():
    global A, lock
    lock.acquire() #開始lock
    for i in range(10):
        A += 1
        print('job1', A)
    lock.release() #結束lock

def job2():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 10
        print('job2', A)
    lock.release()

if __name__ == '__main__':
    lock = threading.Lock()
    A = 0  #共享內存,全局變量
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()


job1job2  201
job2
 job130 
2job2
 job140 
3job2
 job150 
4job2
 job160 
5job2
 job170 
6job2
 job180 
7job2
 job190 
8job2
 job1100 
9job2
 job1110 
10
